In [3]:
import os
import darknet
import pydarknet

from PIL import Image as PILImage

In [2]:
# Darknet is stupid... so all paths are relative at the moment.
os.chdir("..")

In [3]:
# Load the ImageNet 1k labels/metadata
imagenet = darknet.Metadata("cfg/imagenet1k.data")

In [4]:
# Load the Classifier
n = pydarknet.ImageClassifier(labels=imagenet.classes,
                              config_file="cfg/darknet53_448.cfg",
                              weights_file="weights/darknet53_448.weights")

In [5]:
img_filename = "data/dog.jpg"
n.classify("data/dog.jpg", top=5)


[('malamute', 0.9832896),
 ('Eskimo dog', 0.0043422417),
 ('Siberian husky', 0.0032391667),
 ('Tibetan mastiff', 0.0030777778),
 ('Great Pyrenees', 0.0022481713)]

In [6]:
pil_img = PILImage.open(img_filename)
n.classify(pil_img, top=5)

[('malamute', 0.9832896),
 ('Eskimo dog', 0.0043422417),
 ('Siberian husky', 0.0032391667),
 ('Tibetan mastiff', 0.0030777778),
 ('Great Pyrenees', 0.0022481713)]

In [2]:
try:
    import cv2
    # Note: Please note that cv2 nd-arrays are h*w*c ordered.
    cv2_img = cv2.imread(img_filename)
    cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    n.classify(cv2_img, top=5)
except ModuleNotFoundError:
    pass

In [8]:
n.classify("data/eagle.jpg", top=5)

[('bald eagle', 0.55398),
 ('vulture', 0.21862003),
 ('kite', 0.19144633),
 ('ruddy turnstone', 0.004691909),
 ('ruffed grouse', 0.0033115256)]

In [9]:
import requests
majesty_url="https://s3-us-west-2.amazonaws.com/static.music-man.com/website/images/instruments/instrument-77.png?1588624445"
img = PILImage.open(requests.get(majesty_url, stream=True).raw)
n.classify(img, top=5)

[('electric guitar', 0.9876499),
 ('acoustic guitar', 0.009498796),
 ('banjo', 0.0011737668),
 ('pick', 0.00072627096),
 ('stage', 0.0005768967)]